Rather than using cell2cell's internal scoring functions, external tools (e.g. CellChat, CellPhoneDB, etc) can first be run to get communication scores. Once communication scores are attained for each context, they can be reformatted into the 4D-Communication Tensor structure used by Tensor-cell2cell. 

In this tutorial, we will demonstrate how to build the 4D-Communication Tensor from the output of external tools. We assume the user has already run their preferred external tool on each context separately. For robust, easy implementaiton of any tool of interest, we recommend using LIANA (https://github.com/saezlab/liana). You can see how we used LIANA to run multiple tools on the BALF dataset in our CodeOcean capsule (https://doi.org/10.24433/CO.0051950.v2) under code/CCC-Tools/ section. Here, we use the output of CellChat on the BALF dataset as a starting point, which can be found in the CodeOcean under data/CCC-tools/*_cellchat.csv.   

In [10]:
import os
import pandas as pd

In [24]:
# load the data
out_path = '/data3/hratch/c2c_general/'
in_path = '/data3/hratch/c2c_general/cellchat_balf/inputs/'
os.listdir(in_path)

['C100_communication_scores_cellchat.csv',
 'C141_communication_scores_cellchat.csv',
 'C142_communication_scores_cellchat.csv',
 'C143_communication_scores_cellchat.csv',
 'C144_communication_scores_cellchat.csv',
 'C145_communication_scores_cellchat.csv',
 'C146_communication_scores_cellchat.csv',
 'C148_communication_scores_cellchat.csv',
 'C149_communication_scores_cellchat.csv',
 'C152_communication_scores_cellchat.csv',
 'C51_communication_scores_cellchat.csv',
 'C52_communication_scores_cellchat.csv']

In [16]:
# generate a dictionary of communication scores for each sample
scores = {fn.split('_')[0]: pd.read_csv(os.path.join(in_path, fn)) for fn in os.listdir(in_path)}

Many tools outputs are formatted as an adjacency list. Each row delineats the cell pair interacting, the ligand-receptor pair mediating the interaction, and the communication score ("prob" below) for that cell pair. Some tools will also have a p-value which you can use to further filter interactions if desired. 

In [20]:
scores['C51'].head()

,source,target,ligand,receptor,prob,pval
0,Macrophages,Macrophages,JAG2,NOTCH1,0.0,1.0
1,mDC,Macrophages,JAG2,NOTCH1,0.0,1.0
2,T,Macrophages,JAG2,NOTCH1,0.0,1.0
3,NK,Macrophages,JAG2,NOTCH1,0.0,1.0
4,B,Macrophages,JAG2,NOTCH1,0.0,1.0


cell2cell has a function to convert a dictionary of adjacency lists, one for each context, into a 4D-communication tensor:

In [23]:
# from cell2cell.tensor.external_scores import dataframes_to_tensor #https://github.com/earmingol/cell2cell/blob/new_version/cell2cell/tensor/external_scores.py

In [36]:
# tensor_internal = c2c.io.load_variable_with_pickle(out_path + 'tensor_internal.pkl')
# context_order = tensor_internal.order_names[0]

context_order = ['C148', 'C100', 'C142', 'C51', 'C146', 'C52', 'C149', 'C143', 'C152', 'C145', 'C144', 'C141']
tensor = dataframes_to_tensor(context_df_dict = scores, 
                    sender_col = 'source', 
                    receiver_col = 'target', 
                    ligand_col = 'ligand', 
                    receptor_col = 'receptor', 
                    score_col = 'prob', 
                    context_order = context_order)

Save the tensor

In [38]:
c2c.io.export_variable_with_pickle(tensor, out_path + 'tensor_external.pkl')

/data3/hratch/c2c_general/tensor_external.pkl  was correctly saved.
